# Tool Calling in LangChain and LangGraph

This notebook provides a detailed demonstration of the steps and code required to showcase support for Chat models, including the integration of tools using LangGraph and watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Goal 
The purpose of this notebook is to demonstrate how to use Chat models, e.g. `mistralai/mistral-medium-2505` using the tools available in LangChain and LangGraph. LangGraph is an Agent Orchestrator with which you can build graph applications that automatically execute sequences of actions and in which the LLM is the key decision maker that determines the next step. This notebook provides the steps to create a market research agent consisting of various tools such as DuckDuckGo, Wikipedia, and GoogleTrends. 

## Table of Contents 
 
This notebook contains the following parts:

- [Environment Setup](#environment_setup)
- [Foundation Models on watsonx](#fm_setup)
- [LangChain integration](#langchain_integration)
- [Examples](#examples)
- [Summary](#summary)

<a id="environment_setup"></a>
## Environment Setup


### Watsonx.ai Runtime Service
Before you use the sample code in this notebook, ensure an instance of the watsonx.ai Runtime Service is created and associated to the project. The lab guide provides more information on how to associate the watsonx.ai Runtime Service to the watsonx.ai project.

### Install and import the `datasets` and dependencies

In [ ]:
!pip install -U "langgraph>0.2,<0.3" | tail -n 1
!pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1
!pip install ddgs

!pip install langchain wikipedia
!pip install pytrends

In [ ]:
# Needed for DuckDuckGo search tool
import ddgs 

from ddgs import DDGS

## Needed for Wikipedia Tool
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

###
from pytrends.request import TrendReq
from langchain.chat_models import ChatOpenAI # remove this?

## Needed to create tools in LangChain
from langchain.tools import tool

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [ ]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="fm_setup"></a>
## Set up a Foundation Model on `watsonx.ai`

In [ ]:
model_id = "mistralai/mistral-medium-2505"

<a id="langchain_integration"></a>

## LangChain integration

`ChatWatsonx` is a wrapper around watsonx.ai models that provides chat integration around the models.

### Initialize the `ChatWatsonx` class

In [ ]:
from langchain_ibm import ChatWatsonx

chat = ChatWatsonx(
    url=credentials["url"],
    apikey=credentials["apikey"],
    model_id=model_id,
    project_id=project_id,
)

### Define the assistant's tools 

In [ ]:
from langchain_core.tools import tool

@tool
def duckduckgo_search(query: str) -> str:
    """Use this tool to search the web using DuckDuckGo. Input should be a search query."""    
    with DDGS() as ddgs:
        results = ddgs.text(query)
        if results:
            return results[0]['body']  # Return the snippet of the first result
        else:
            return "No results found."

@tool
def wikipedia_tool(query: str) -> str:
    """Useful for answering questions about general knowledge or historical topics using Wikipedia."""    
    wiki = WikipediaAPIWrapper()
    func=WikipediaQueryRun(api_wrapper=wiki).run


@tool
def get_google_trends(keyword: str) -> str:
    "Use this tool to get Google search trend data for a specific keyword over the past year."
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([keyword], cat=0, timeframe='today 12-m', geo='', gprop='')
    interest_over_time_df = pytrends.interest_over_time()

    if interest_over_time_df.empty:
        return f"No trend data found for '{keyword}'."
    
    trend_summary = interest_over_time_df[keyword].describe()
    return (
        f"Google Trends data for '{keyword}':\n"
        f"- Mean interest: {trend_summary['mean']:.2f}\n"
        f"- Max interest: {trend_summary['max']}\n"
        f"- Min interest: {trend_summary['min']}\n"
        f"- Data points: {len(interest_over_time_df)}"
    )

tools = [duckduckgo_search, wikipedia_tool, get_google_trends]

### Define the graph.

In [ ]:
from langgraph.prebuilt import create_react_agent

graph = create_react_agent(chat, tools=tools)

### Visualize the graph you just created.

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

### Add a helpful function to print messages from the model.

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

<a id="examples"></a>

## Examples

Run the app with an input that needs a tool call. 

<b>Note:</b> Results may differ each time the code is run.

### Example 1 - DuckDuckGo Tool

In [ ]:
inputs = {"messages": [("user", "What is the latest news about AI regulation?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

### Example 2 - DuckDuckGo Tool

In [ ]:
inputs = {"messages": [("user", "What are the latest news headlines about IBM?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

### Example 3 - DuckDuckGo or Wikipedia Tool Example
<b>Note:</b> Results may differ each time the code is run - sometimes the agent may use the Wikipedia tool, and sometimes it may use the DuckDuckGo search tool. 

In [ ]:
inputs = {"messages": [("user", "Who is the CEO of IBM?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

### Example 4 - Google Trends Tool

<b>Note:</b> You may receive an error related to rate limits, or it may default to another tool. If a rate limit error occurs consider waiting some time and rerunning the code cell. This example highlights the importance of considering usage limitations and API requirements when building tools for agents, especially in production scenarios. Additionally, this is another example where the agent may use a different tool other than Google Trends Tool.

In [ ]:

inputs = {"messages": [("user", "How popular is electric vehicles?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

### Example 5 - Google Trends Tool 
<b>Note:</b> You may receive an error related to rate limits, or it may default to another tool. If a rate limit error occurs consider waiting some time and rerunning the code cell. This example highlights the importance of considering usage limitations and API requirements when building tools for agents. 

In [ ]:
inputs = {"messages": [("user", "Which countries search for 'machine learning' the most?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

<a id="summary"></a>

## Summary and next steps

You successfully completed this notebook!

You learned how to build a simple graph using LangGraph and `ChatWatsonx`.

Check out the <a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

This notebook was modified from the original notebook created by **Mateusz Szewczyk**, Software Engineer at watsonx.ai. View the <a href="https://github.com/IBM/watsonx-ai-samples/blob/master/cloud/notebooks/python_sdk/deployments/foundation_models/chat/Use%20watsonx%2C%20and%20%60mistral-large%60%20with%20support%20for%20tools%20to%20perform%20simple%20calculations.ipynb">original notebook here</a>.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.